-----
## SMA and RSI into the strategy

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
# Import the finta library
from finta import TA

In [2]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
blk_df = pd.read_csv(
    Path("../Resources/blk_ohlcv.csv"),
    index_col = "date", 
    parse_dates = True, 
    # infer_datetime_format = True
)

# Review the DataFrame
blk_df.head()

,open,high,low,close,volume
date,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135
2018-01-03,508.66,514.74,508.16,514.34,448724
2018-01-04,516.50,521.31,515.01,521.13,389830
2018-01-05,523.86,525.94,520.89,525.57,482968
2018-01-08,523.94,529.72,523.52,529.49,561517


### Calculate SMA and RSI
First, calculate the SMA for your chosen windows (short and long) and the RS window of 14 time..

In [3]:
# Select your windows
short_window = 50
long_window = 100

In [4]:
# Calculate the SMA
blk_df['Short_SMA'] = TA.SMA(blk_df, short_window)
blk_df['Long_SMA'] = TA.SMA(blk_df, long_window)

# Calculate the RSI using FINTA library
# In the context of Relative Strength Index (RSI), a window of 14 means that the RSI is calculated over a period of 14 consecutive data points. 
# The RSI is a momentum oscillator that measures the speed and change of price movements. It ranges from 0 to 100 and is typically used to identify overbought or oversold conditions in a security. 
# A common practice is to use a 14-period RSI, which means that it calculates the average gain and average loss over the last 14 periods (e.g., 14 days for daily data or 14 hours for hourly data) to determine the RSI value at each point in time.

blk_df['RSI'] = TA.RSI(blk_df, 14)  

# Display the DataFrame with RSI
blk_df

,open,high,low,close,volume,Short_SMA,Long_SMA,RSI
date,,,,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135,NaN,NaN,NaN
2018-01-03,508.66,514.74,508.16,514.34,448724,NaN,NaN,100.000000
2018-01-04,516.50,521.31,515.01,521.13,389830,NaN,NaN,100.000000
2018-01-05,523.86,525.94,520.89,525.57,482968,NaN,NaN,100.000000
2018-01-08,523.94,529.72,523.52,529.49,561517,NaN,NaN,100.000000
...,...,...,...,...,...,...,...,...
2020-12-24,703.36,710.84,703.36,707.18,173253,671.8904,624.9612,58.307304
2020-12-28,714.59,715.12,709.07,710.18,272253,673.1692,626.2415,59.590040
2020-12-29,713.74,715.32,705.61,706.28,318436,674.1472,627.4820,57.129309


### Generate Trading Signals
You'll then need to define your criteria for when a buy or sell signal is triggered based on the SMA and RSI. For example, a common approach might be to buy when the short SMA crosses above the long SMA (a "golden cross") and the RSI is below a certain threshold, indicating that the asset might be oversold. Conversely, you might sell when the short SMA crosses below the long SMA (a "death cross") and the RSI is above a certain threshold, indicating that the asset might be overbought.

In [11]:
# Initialize the signal with zero
blk_df['Signal'] = 0.0

# Generate signals based on SMA crossover and RSI
# Here as an example: Buy signal (1) when short SMA is above long SMA and RSI is below 30 (oversold)
# Sell signal (-1) when short SMA is below long SMA and RSI is above 70 (overbought)
blk_df.loc[(blk_df['Short_SMA'] > blk_df['Long_SMA']) & (blk_df['RSI'] < 30), 'Signal'] = 1
blk_df.loc[(blk_df['Short_SMA'] < blk_df['Long_SMA']) & (blk_df['RSI'] > 70), 'Signal'] = -1

# Calculate the points in time at which a position should be taken, 1 or -1
blk_df['Entry/Exit'] = blk_df['Signal'].diff()
blk_df.tail(50)

,open,high,low,close,volume,Short_SMA,Long_SMA,RSI,Signal,Entry/Exit,Middle Band,Upper Band,Lower Band
date,,,,,,,,,,,,,
2020-10-21,646.58,653.72,634.82,634.99,452696,584.0152,572.1844,63.568536,0.0,0.0,598.9715,675.920861,522.022139
2020-10-22,635.00,637.83,629.82,635.18,483384,584.9140,573.1228,63.626022,0.0,0.0,603.9780,676.306950,531.649050
2020-10-23,637.87,639.16,633.51,637.31,494217,585.8690,574.0188,64.305996,0.0,0.0,608.4530,677.132548,539.773452
2020-10-26,630.00,631.59,615.83,622.19,686099,586.5436,574.7788,56.265209,0.0,0.0,611.6300,676.375138,546.884862
2020-10-27,620.32,624.58,613.47,614.53,349823,586.9936,575.3588,52.671953,0.0,0.0,614.6165,673.576002,555.656998
2020-10-28,602.69,606.45,596.26,600.71,645648,587.2236,575.7926,46.857710,0.0,0.0,616.4745,670.819632,562.129368
2020-10-29,600.21,606.59,590.26,602.33,447449,587.4752,576.2353,47.588067,0.0,0.0,618.1955,668.060450,568.330550
2020-10-30,597.58,600.87,587.90,599.21,530463,587.7922,576.6468,46.269169,0.0,0.0,619.6500,665.113281,574.186719
2020-11-02,608.96,618.35,606.45,614.87,455350,588.4088,577.5434,53.269738,0.0,0.0,621.3445,663.115492,579.573508


### Adjust According to Your Strategy
SMA Condition: The condition (blk_df['Short_SMA'] > blk_df['Long_SMA']) checks for the golden cross, but you can adjust the logic based on your strategy.
RSI Thresholds: The values 30 and 70 are common thresholds for oversold and overbought conditions, respectively, but these can be adjusted based on your analysis and risk tolerance.
Experiment with different conditions and thresholds to see what works best for your trading strategy

## Visualize the results

In [6]:
# Visualize entry position relative to close price
entry = blk_df[blk_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = blk_df[blk_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="orange",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = blk_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = blk_df[["Short_SMA", "Long_SMA"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

To add the RSI (Relative Strength Index) to your graph and visualize it alongside your current plots, you would typically plot it on a separate axis because it operates on a different scale (usually 0 to 100)  compared to price. However, integrating it directly into the same plot could make it challenging to interpret due to the scale differences. Therefore, createng a separate plot for the RSI and then displaying it below or above the price chart for comparative analysin:

In [7]:
# Visualize the RSI
rsi_plot = blk_df['RSI'].hvplot(
    line_color='blue',
    ylabel='RSI',
    width=1000,
    height=200,
    ylim=(0, 100)  # RSI ranges from 0 to 100
)

# Overlay the entry and exit on the price chart as before
entry_exit_plot = security_close * moving_avgs * entry * exit

# Combine the price chart and RSI chart for display
final_plot = entry_exit_plot + rsi_plot

final_plot.cols(1)  # Organize the plots vertically


:Layout
   .Overlay.I :Overlay
      .Curve.I          :Curve   [date]   (close)
      .NdOverlay.I      :NdOverlay   [Variable]
         :Curve   [date]   (value)
      .Scatter.Close.I  :Scatter   [date]   (close)
      .Scatter.Close.II :Scatter   [date]   (close)
   .Curve.RSI :Curve   [date]   (RSI)

------
## Adding Bollinger Bands to SMA and RSI Strategy

#### In this strategy:

* Buy signals (1) are generated when the short SMA is above the long SMA, RSI is below 30 (oversold), and the price is below the lower Bollinger Band.
* Sell signals (-1) are generated when the short SMA is below the long SMA, RSI is above 70 (overbought), and the price is above the upper Bollinger Band.

You can adjust the parameters and conditions according to your specific trading strategy and risk tolerance.
nce.

In [8]:
# Calculate Bollinger Bands using FINTA library
bollinger_bands = TA.BBANDS(blk_df, period=20)

# Extracting bands from the result
blk_df['Middle Band'] = bollinger_bands['BB_MIDDLE']
blk_df['Upper Band'] = bollinger_bands['BB_UPPER']
blk_df['Lower Band'] = bollinger_bands['BB_LOWER']

# Display the DataFrame with Bollinger Bands
display(blk_df)

# Initialize the signal with zero
blk_df['Signal'] = 0.0

# Generate signals based on SMA crossover, RSI, and Bollinger Bands
# Buy signal (1) when short SMA is above long SMA, RSI is below 30 (oversold), and price is below lower Bollinger Band
# Sell signal (-1) when short SMA is below long SMA, RSI is above 70 (overbought), and price is above upper Bollinger Band
blk_df.loc[(blk_df['Short_SMA'] > blk_df['Long_SMA']) & (blk_df['RSI'] < 30) & (blk_df['close'] < blk_df['Lower Band']), 'Signal'] = 1
blk_df.loc[(blk_df['Short_SMA'] < blk_df['Long_SMA']) & (blk_df['RSI'] > 70) & (blk_df['close'] > blk_df['Upper Band']), 'Signal'] = -1

# Calculate the points in time at which a position should be taken, 1 or -1
blk_df['Entry/Exit'] = blk_df['Signal'].diff()
blk_df.iloc[95:105, :]


,open,high,low,close,volume,Short_SMA,Long_SMA,RSI,Signal,Entry/Exit,Middle Band,Upper Band,Lower Band
date,,,,,,,,,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2018-01-03,508.66,514.74,508.16,514.34,448724,NaN,NaN,100.000000,0.0,0.0,NaN,NaN,NaN
2018-01-04,516.50,521.31,515.01,521.13,389830,NaN,NaN,100.000000,0.0,0.0,NaN,NaN,NaN
2018-01-05,523.86,525.94,520.89,525.57,482968,NaN,NaN,100.000000,0.0,0.0,NaN,NaN,NaN
2018-01-08,523.94,529.72,523.52,529.49,561517,NaN,NaN,100.000000,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,703.36,710.84,703.36,707.18,173253,671.8904,624.9612,58.307304,0.0,0.0,703.6545,721.456103,685.852897
2020-12-28,714.59,715.12,709.07,710.18,272253,673.1692,626.2415,59.590040,0.0,0.0,703.4080,720.670064,686.145936
2020-12-29,713.74,715.32,705.61,706.28,318436,674.1472,627.4820,57.129309,0.0,0.0,703.8045,720.941228,686.667772


,open,high,low,close,volume,Short_SMA,Long_SMA,RSI,Signal,Entry/Exit,Middle Band,Upper Band,Lower Band
date,,,,,,,,,,,,,
2018-05-18,536.28,539.92,534.42,538.49,518719,533.7828,NaN,57.051309,0.0,0.0,525.5990,544.911923,506.286077
2018-05-21,542.00,547.98,542.00,543.48,359081,533.1088,NaN,60.279613,0.0,0.0,526.7805,547.455723,506.105277
2018-05-22,546.44,549.58,545.86,546.88,406234,532.6106,NaN,62.355887,0.0,0.0,528.4395,550.001176,506.877824
2018-05-23,541.72,542.79,533.75,542.66,377715,532.1826,NaN,58.283623,0.0,0.0,529.9615,550.993424,508.929576
2018-05-24,542.94,542.94,534.68,540.83,413054,531.8200,540.7656,56.558655,0.0,0.0,530.8670,552.144168,509.589832
2018-05-25,537.14,540.32,535.94,539.61,290043,531.4536,541.0720,55.381875,0.0,0.0,531.6150,553.023355,510.206645
2018-05-29,534.75,537.82,518.03,524.26,714247,530.6656,541.1712,43.202206,0.0,0.0,531.7530,552.921053,510.584947
2018-05-30,527.31,535.29,526.07,534.29,404142,530.2748,541.3028,50.813993,0.0,0.0,532.5690,552.734416,512.403584
2018-05-31,527.31,535.29,526.07,534.29,83,529.8102,541.3900,50.813993,0.0,0.0,533.5440,551.893906,515.194094


In [9]:
# Visualize Bollinger Bands
bollinger_bands_plot = blk_df[['Upper Band', 'Middle Band', 'Lower Band']].hvplot(
    ylabel='Price in $',
    line_width=2,
    width=1000,
    height=200,
    color=['red', 'green', 'red']  # Upper and lower bands in red, middle band in green
)

# Overlay the entry and exit on the price chart as before
entry_exit_plot = security_close * moving_avgs * entry * exit

# Combine the price chart, RSI chart, and Bollinger Bands chart for display
final_plot = entry_exit_plot + rsi_plot + bollinger_bands_plot

final_plot.cols(1)  # Organize the plots vertically


:Layout
   .Overlay.I   :Overlay
      .Curve.I          :Curve   [date]   (close)
      .NdOverlay.I      :NdOverlay   [Variable]
         :Curve   [date]   (value)
      .Scatter.Close.I  :Scatter   [date]   (close)
      .Scatter.Close.II :Scatter   [date]   (close)
   .Curve.RSI   :Curve   [date]   (RSI)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [date]   (value)

In [10]:
# Overlay the entry and exit on the price chart along with Bollinger Bands
entry_exit_plot_with_bands = entry_exit_plot * bollinger_bands_plot

# Combine the price chart, RSI chart, and Bollinger Bands chart for display
final_plot = entry_exit_plot_with_bands + rsi_plot

final_plot.cols(1)  # Organize the plots vertically

:Layout
   .Overlay.I :Overlay
      .Curve.I          :Curve   [date]   (close)
      .NdOverlay.I      :NdOverlay   [Variable]
         :Curve   [date]   (value)
      .Scatter.Close.I  :Scatter   [date]   (close)
      .Scatter.Close.II :Scatter   [date]   (close)
      .NdOverlay.II     :NdOverlay   [Variable]
         :Curve   [date]   (value)
   .Curve.RSI :Curve   [date]   (RSI)

### Important Considerations:
1. **Strategy Complexity:**  
   Integrating multiple indicators like SMAs, EMAs, RSI, and Bollinger Bands increases the complexity of your trading strategy. It's crucial to understand how each indicator works and how they can complement each other.

2. **Parameter Selection:**  
   The window sizes for the SMAs, EMAs, RSI, and Bollinger Bands (including the standard deviation multiplier for the bands) can significantly impact your strategy's performance. It may require extensive backtesting to optimize these parameters.

3. **Market Conditions:**  
   Different market conditions may require adjustments to your strategy. For instance, in a highly volatile market, the price might frequently touch the Bollinger Bands, leading to potential false signals.

4. **Backtesting:**  
   Always backtest your strategy on historical data to understand its potential performance and to adjust parameters before considering live trading.

By combining these indicators, you can create a strategy that potentially capitalizes on both trend following (via EMAs) and volatility (via Bollinger Bands), but always ensure thorough backtesting.
ghly.